In [35]:
import numpy as np

In [36]:
# Security paprameter
epsSEC = 10**-10

# Error correction parameter (Failure probability)
epsCOR = 10**-10

f = 1.18 # error correction efficiency

In [56]:
file_name = "3second_data_QKDkeyextraction_6_HWP=28_3data_50pscw_4.5degreeincrement.txt"
#file_name = "10second_data_QKDkeyextraction_4_HWP=28_10data_50pscw.txt"


data = None
with open(file_name, "r") as f:
    data = np.loadtxt(f)
    

In [57]:
print(data)

[[205961. 206954. 208199.]
 [  3192.   2991.   2917.]
 [  2929.   3055.   3171.]
 [201423. 195461. 200092.]
 [193944. 201508. 205015.]
 [  5396.   4402.   5476.]
 [  5629.   6812.   7661.]
 [186904. 186948. 186740.]]


In [58]:
counts = np.sum(data, axis=1)
counts #DataOrder: HH HV VH VV DD DA AD AA

array([621114.,   9100.,   9155., 596976., 600467.,  15274.,  20102.,
       560592.])

In [59]:
# Raw Key Length
raw_key_length = np.sum(counts)

int(raw_key_length)

2432780

In [60]:
#QBER = (HV+VH+AD+DH)/(HH+VV+DD+AA+HV+VH+AD+DH)

QBER = (counts[1]+counts[2]+counts[5]+counts[6])/(raw_key_length)
print("QBER", QBER)

QBER 0.02204514999301211


In [61]:
#binary entorpy function
def h(p):
    if p == 1: return 0
    if p == 0: return 0
    return -p*np.log2(p)-(1-p)*np.log2(1-p)

In [62]:
#finite key analysis using: https://doi.org/10.1038/ncomms1631

def compute_l(N,beta,qber,f): 
    q=1
    Qtol = qber
    k = beta*N
    n = N - k
    #print ("k",k,"n",n)
    term1 = (n+k)/(n*k)
    term2 = (k+1)/k
    term3 = np.log(2/epsSEC)
    interm = term1*term2*term3
    mu = np.sqrt(interm)
    error_term = np.log2(2/(epsSEC*epsSEC*epsCOR))
    leakEC = f*n*h(qber)
    if (Qtol+mu >0.11 or Qtol+mu <0 ):
        
        return 0
    else:
        #print("here good mu",mu)
        l = n*(q-h(Qtol+mu) ) - leakEC -error_term 
        return l


In [63]:
#compute_l(93734,0.10,0.0439)

In [64]:
beta_vals = np.linspace(0.0005005,.5,500)

In [65]:
def optimize(N,beta_vals,qber,f):
    max_l = 0
    for beta in beta_vals:
        #print ("beta",beta)
        l = compute_l(N,beta,qber,f)
        if max_l < l:
            #print("here",l)
            max_l = l
            #print ("maxl",int(max_l))
    return max_l

In [68]:
# Divide raw_key_length by 2, to account for 3db loss due to mismatch basis in the 50:50 beam splitter
secret_key_length=optimize(raw_key_length/2,beta_vals,QBER,f=1.18) 
print ("secret_key_length",int(secret_key_length))
print ("raw_key_length",int(raw_key_length/2))
print ("QBER",QBER)

secret_key_length 656882
raw_key_length 1216390
QBER 0.02204514999301211


# Results

## 10 sec data

secret_key_length 1474871\
raw_key_length 3715295\
QBER 0.0403216972003569

## 3 sec data

secret_key_length 656882\
raw_key_length 1216390\
QBER 0.02204514999301211

## Notes
Please report the pump rate, and total integration time, and normalize properly to get a key_rate that is, secret_key_bits/second count that are comparable to each other. 
